In [ ]:
import geopandas as gpd
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, GeoJSON
from ipywidgets import Output, SelectionSlider, VBox, Image, RadioButtons, Text, Button, Layout
from pathlib import Path
import numpy as np
import pyproj
from IPython.display import display #, Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from io import BytesIO
from scipy.spatial import distance_matrix

In [2]:
path_project = Path().absolute()

In [3]:
path_save_merged = path_project / "files/Europe/Europe_IGNF_grid_50km_intersection_tiles_centroid_BT.shp"
merged_tiles_centroid = gpd.read_file(str(path_save_merged))

path_gdf_part = path_project /"files/Europe/Europe_IGNF_grid_50km_all_albedo_final_BT.shp"
gdf = gpd.read_file(path_gdf_part)

if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs('EPSG:4326')
nb_tiles_y = 90 
nb_tiles_y_French_Alps = 72
nb_tiles_y_europe = 90 

In [4]:
path_save_merged_bis= path_project / "files/French_Alps/French_Alps_IGNF_grid_5km_intersection_tiles_centroid_BT.shp"
merged_tiles_centroid_bis = gpd.read_file(str(path_save_merged_bis))


path_gdf_french_alps = path_project / "files/French_Alps/French_Alps_IGNF_grid_5km_all_albedo_final_BT.shp"
gdf_bis = gpd.read_file(path_gdf_french_alps).to_crs(epsg=4326)
if gdf_bis.crs != 'EPSG:4326':
    gdf_bis = gdf_bis.to_crs('EPSG:4326')

style_europe = {'color': 'black', 'weight': 2, 'fillOpacity': 0}
style_french_alps = {'color': 'blue', 'weight': 2, 'fillOpacity': 0}


In [5]:
path_centroid_Europe = path_project / "files/Europe/Europe_IGNF_extent_centroid_coordinates.shp"
centroid_Europe = gpd.read_file(str(path_centroid_Europe))
center_Europe = [centroid_Europe.loc[0,'latitude'], centroid_Europe.loc[0,'longitude']]
zoom_level = 4
layout = Layout(width='1000px', height='800px')
m = Map(center=center_Europe, zoom=zoom_level, basemap=basemap_to_tiles(basemaps.CartoDB.Positron), layout=layout)

In [6]:
geo_json_layer_europe = GeoJSON(data=gdf.__geo_interface__, style=style_europe)
geo_json_layer_french_alps = GeoJSON(data=gdf_bis.__geo_interface__, style=style_french_alps)
m.add_layer(geo_json_layer_europe)

In [7]:
global_coords = None
best_tilt_interpolation = None
gcr_value = 0.2
albedo_value = 0.2
id = None

In [8]:
def on_area_change(change):
    global current_tiles_centroid, nb_tiles_y
    if change['new'] == 'French Alps':
        m.remove_layer(geo_json_layer_europe)
        m.add_layer(geo_json_layer_french_alps)
        nb_tiles_y =nb_tiles_y_French_Alps
        current_tiles_centroid = merged_tiles_centroid_bis
    elif change['new'] == 'Europe':
        m.remove_layer(geo_json_layer_french_alps)
        m.add_layer(geo_json_layer_europe)
        current_tiles_centroid = merged_tiles_centroid
        nb_tiles_y = nb_tiles_y_europe
    if global_coords:
        handle_interaction(type='click', coordinates=global_coords)
current_tiles_centroid = merged_tiles_centroid

In [9]:
message_graph= Output()
def display_message_graph():
    message_graph.clear_output()
    with message_graph:
        print(f"Select the parameters (Ground Coverage Ratio (GCR) and albedo), the scalebar normalization and the area \nfor which the map of optimal tilt angle will be displayed :")
display_message_graph()

In [10]:
message= Output()
def display_initial_message():
    message.clear_output()
    with message:
        print(f"Click on a location on the map or enter the coordinates to know the optimal tilt angle for south-facing bifacial PV plant \nfor an albedo value of {albedo_value} and Ground Coverage Ratio of {gcr_value}:")
display_initial_message()

In [11]:
slider_options = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7]
slider_GCR = SelectionSlider(
    options=slider_options,
    value=0.2, 
    description='GCR:', 
    continuous_update=False, 
    orientation='horizontal', 
    readout=True 
)
def on_slider_change_GCR(change):
    global gcr_value
    gcr_value = slider_GCR.value
    display_initial_message()
    if global_coords:
        handle_interaction(type='click', coordinates=global_coords)
slider_GCR.observe(on_slider_change_GCR, names='value')

In [12]:
slider_options_albedo = [0.1, 0.2, 0.3, 0.4, 0.5]
slider_albedo = SelectionSlider(
    options=slider_options_albedo,
    value=0.2,
    description='Albedo:', 
    continuous_update=False, 
    orientation='horizontal',  
    readout=True  
)
def on_slider_change_albedo(change):
    global albedo_value
    albedo_value = slider_albedo.value
    display_initial_message()
    if global_coords:
        handle_interaction(type='click', coordinates=global_coords)
slider_albedo.observe(on_slider_change_albedo, names='value')

In [13]:
def transform_coordinates(lat, lon, target_proj='IGNF:ETRS89LAEA'):
    wgs84 = pyproj.CRS('EPSG:4326')
    target = pyproj.CRS(target_proj)
    transformer = pyproj.Transformer.from_crs(wgs84, target, always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

In [14]:
def find_id_tiles(x, y,tiles_centroid ): 
    id = 0
    for i in range(0, len(tiles_centroid)) : 
        top =  tiles_centroid.loc[i, 'top']
        left = tiles_centroid.loc[i, 'left']
        right = tiles_centroid.loc[i, 'right']
        bottom = tiles_centroid.loc[i, 'bottom']
        if x > left and x < right and y< top and y > bottom : 
            id = tiles_centroid.loc[i, "id"]
            break
    return id

In [15]:
def find_quarter(id, coords, tiles_centroid) : 
    index = tiles_centroid[tiles_centroid['id']==id].index
    latitude = tiles_centroid.loc[index, 'latitude']
    longitude = tiles_centroid.loc[index, 'longitude']
    x_center, y_center = transform_coordinates(latitude, longitude, target_proj='IGNF:ETRS89LAEA')

    lat_point = coords[0]
    long_point = coords[1]
    x_point, y_point = transform_coordinates(lat_point, long_point, target_proj='IGNF:ETRS89LAEA')

    top = False
    right = False
    if y_point >  y_center : 
        top = True

    if x_point > x_center : 
        right = True
    
    position = ""
    if top== True and right == True : 
        position = "top right"
    if top == True and right == False : 
        position = "top left"
    if top == False and right == True : 
        position = "bottom right"
    if top == False and right == False: 
        position = "bottom left"

    return index, top, right, position, latitude, x_center

In [16]:
def find_id_voisin(id, position, nb_tiles_y):
    list_id_voisin = []
    list_id_voisin.append(id)
    if position == "top right": 
        list_id_voisin.append(id+nb_tiles_y)
        list_id_voisin.append(id+nb_tiles_y-1)
        list_id_voisin.append(id-1)
    if position == "top left": 
        list_id_voisin.append(id-nb_tiles_y)
        list_id_voisin.append(id-nb_tiles_y-1)
        list_id_voisin.append(id-1) 
    if position == "bottom right": 
        list_id_voisin.append(id+nb_tiles_y)
        list_id_voisin.append(id+nb_tiles_y+1)
        list_id_voisin.append(id+1)
    if position == "bottom left": 
        list_id_voisin.append(id-nb_tiles_y)
        list_id_voisin.append(id-nb_tiles_y+1)
        list_id_voisin.append(id+1) 

    return list_id_voisin

In [17]:
def obtain_list_coordinates_and_values(list_id_voisin, tiles_centroid, GCR = "02", ALBEDO = "02"):
    list_index = []
    list_best_tilt = []
    list_coordinates = []
    for id in list_id_voisin :
        filtered_df = tiles_centroid[tiles_centroid['id'] == id]
        if not filtered_df.empty:
            index_temp = filtered_df.index[0]
            list_index.append(index_temp)
            latitude_temp = filtered_df.loc[index_temp, 'latitude']
            longitude_temp = filtered_df.loc[index_temp, 'longitude']
            x_temp, y_temp =  transform_coordinates(latitude_temp, longitude_temp, target_proj='IGNF:ETRS89LAEA')
            list_coordinates.append([x_temp, y_temp])
            list_best_tilt.append(filtered_df.loc[index_temp, f'BT_{GCR}_{ALBEDO}'])
        else : 
            list_index.append(np.nan)
            list_coordinates.append([np.nan, np.nan])
            list_best_tilt.append(np.nan)
    return list_coordinates, list_index, list_best_tilt

In [18]:
def idw_interpolation(list_coordinates, x_point, y_point, list_best_tilt, power=2):

    list_coords_clean = [coord for i, coord in enumerate(list_coordinates) if not np.isnan(list_best_tilt[i])]
    list_best_tilt_clean = [value for value in list_best_tilt if not np.isnan(value)]
    list_coords_clean = np.array(list_coords_clean)
    list_best_tilt_clean = np.array(list_best_tilt_clean)
    unknown_point = [x_point, y_point]
    unknown_point = np.array(unknown_point).reshape(1, -1)
    distances = distance_matrix(list_coords_clean, unknown_point).flatten()
    distances[distances == 0] = 1e-10
    weights = 1 / np.power(distances, power)
    interpolated_value = np.sum(weights * list_best_tilt_clean) / np.sum(weights)

    return interpolated_value


In [19]:
latitude_input = Text(description="Latitude:")
longitude_input = Text(description="Longitude:")
submit_button = Button(description="Submit")

In [20]:
coord_output = Output()
markers = []
@coord_output.capture()
def handle_interaction(**kwargs):
    global global_coords, albedo_value, id, best_tilt_interpolation, coord_output

    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')

        global_coords = coords
        gcr_value = slider_GCR.value
        albedo_value = slider_albedo.value
        
        if gcr_value != 0.05:
            GCR = f"0{int(gcr_value * 10)}"
        else: 
            GCR = "005"

        ALBEDO = f'0{int(albedo_value*10)}'
        
        x, y = transform_coordinates(coords[0], coords[1])
        id = find_id_tiles(x, y, current_tiles_centroid)
        index, top, right, position, latitude, x_center = find_quarter(id, coords, current_tiles_centroid)
        list_id_voisin = find_id_voisin(id, position, nb_tiles_y)
        list_coordinates, list_index, list_best_tilt = obtain_list_coordinates_and_values(list_id_voisin,current_tiles_centroid,  GCR, ALBEDO)
        if all(np.isnan(item) for item in list_best_tilt) == False: 
            best_tilt_interpolation = idw_interpolation(list_coordinates,x,y, list_best_tilt)
            only_nan = False
        else : 
            only_nan = True 

        coord_output.clear_output()
        with coord_output:
            
            print(f'Coordinates clicked: [{coords[0]:.3f},{coords[1]:.3f}]')
            if only_nan== False : 
                print(f'Best tilt of the tile: {list_best_tilt[0]}')
                print(f'Best tilt interpolated from 4 nearest points : {best_tilt_interpolation:.3f}')
            if only_nan: 
                print(f'The point is out of bounds!')
        for marker in markers:
            m.remove_layer(marker)
        markers.clear()

        marker = Marker(location=coords)
        m.add_layer(marker)
        markers.append(marker)

m.on_interaction(handle_interaction)


In [21]:
@coord_output.capture()
def handle_submit_click(button):
    """Gère le clic sur le bouton Submit."""
    global global_coords, albedo_value, id, best_tilt_interpolation, coord_output

    try:
        lat = float(latitude_input.value)
        lon = float(longitude_input.value)
        coords = [lat, lon]    
        global_coords = coords
        
        # Traitement similaire à ce que vous avez dans le code original pour un clic sur Submit
        gcr_value = slider_GCR.value
        albedo_value = slider_albedo.value

        if gcr_value != 0.05:
            GCR = f"0{int(gcr_value * 10)}"
        else:
            GCR = "005"

        ALBEDO = f'0{int(albedo_value * 10)}'

        x, y = transform_coordinates(coords[0], coords[1])
        id = find_id_tiles(x, y, current_tiles_centroid)
        index, top, right, position, latitude, x_center = find_quarter(id, coords, current_tiles_centroid)
        list_id_voisin = find_id_voisin(id, position, nb_tiles_y)
        list_coordinates, list_index, list_best_tilt = obtain_list_coordinates_and_values(list_id_voisin, current_tiles_centroid, GCR, ALBEDO)

        if all(np.isnan(item) for item in list_best_tilt) == False:
            best_tilt_interpolation = idw_interpolation(list_coordinates, x, y, list_best_tilt)
            only_nan = False
        else:
            only_nan = True

        # Mise à jour de la sortie
        coord_output.clear_output()
        with coord_output:
            print(f'Coordinates entered: [{coords[0]:.3f},{coords[1]:.3f}]')
            if not only_nan:
                print(f'Best tilt of the tile: {list_best_tilt[0]}')
                print(f'Best tilt interpolated from 4 nearest points: {best_tilt_interpolation:.3f}')
            else:
                print(f'The point is out of bounds!')

        # Mise à jour du marqueur sur la carte
        for marker in markers:
            m.remove_layer(marker)
        markers.clear()

        marker = Marker(location=coords)
        m.add_layer(marker)
        markers.append(marker)
    
    except ValueError:
        with coord_output:
            coord_output.clear_output()
            print("Please enter valid numeric values for latitude and longitude.")
        return


submit_button.on_click(handle_submit_click)


In [22]:
def get_image(slider1_value, slider2_value, radio_group1_value, radio_group2_value, radio_group3_value):
    if radio_group1_value == "Optimal tilt angle" :  
        variable = "BT"
    elif radio_group1_value == "Irradiation gain" : 
        variable = "GI"
    else : 
        variable = "Bk"

    if radio_group2_value == "By GCR" : 
        normalisation = "norm_GCR"
    elif radio_group2_value == "By albedo" : 
        normalisation ="norm_albedo"
    elif radio_group2_value == "By all values" : 
        normalisation = "norm_all"
    else : 
        normalisation = "normal"

    GCR_value = str(slider1_value)
    GCR_value = GCR_value[2:]
    albedo_value = str(slider2_value)
    albedo_value = albedo_value[2:]
    norm = normalisation
    if norm == "normal" : 
        norm =""
    else : 
        norm = norm +"_"
    
    if radio_group3_value == "Europe" : 
        img_path = path_project / f'Graphics/Europe/MAP_{variable}/{normalisation}/map_grid_50km_GCR_0{GCR_value}_albedo_0{albedo_value}_{norm}{variable}.png'
    else : 
        img_path = f'Graphics/French_Alps/MAP_{variable}/{normalisation}/map_grid_5km_GCR_0{GCR_value}_albedo_0{albedo_value}_{norm}{variable}_AURA_PACA.png'

    img = mpimg.imread(img_path)

    buffer = BytesIO()
    plt.imsave(buffer, img)
    buffer.seek(0)
    
    return buffer

In [23]:
radio_group1 = RadioButtons(
    options = ['Optimal tilt angle', 'Irradiation gain', 'Irradiation at best tilt'],
    description='Variable',
    disabled=False
)

In [24]:
radio_group2 = RadioButtons(
    options=['By GCR', 'By albedo', 'By all values', 'No normalisation'],
    description='Scalebar normalisation',
    disabled=False
)

In [25]:
radio_group3 = RadioButtons(
    options=['Europe', 'French Alps'],
    description='Area',
    disabled=False
)

In [26]:
layout = Layout(width='1000px') 
image_widget = Image(value=get_image(slider_GCR.value, slider_albedo.value, radio_group1.value, radio_group2.value, radio_group3.value).read(), format='png', layout=layout)


In [27]:
def update_image(*args):
    image_widget.value = get_image(slider_GCR.value, slider_albedo.value, radio_group1.value, radio_group2.value, radio_group3.value).read()


In [28]:
slider_GCR.observe(update_image, names='value')
slider_albedo.observe(update_image, names='value')
radio_group1.observe(update_image, names='value')
radio_group2.observe(update_image, names='value')
radio_group3.observe(update_image, names='value')
radio_group3.observe(on_area_change, names='value')

In [29]:
vbox = VBox([message_graph, slider_GCR, slider_albedo, radio_group1, radio_group2, radio_group3, image_widget, m,message, latitude_input, longitude_input, submit_button, coord_output])

In [30]:
display(vbox)